In [1]:
import os

os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/wkarlina001/end-to-end-ml.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="wkarlina001"
os.environ["MLFLOW_TRACKING_PASSWORD"]="9e92f54fc366d8f0f6d5e6a7adce34c1a1d51177"


In [2]:
import tensorflow as tf
from dataclasses import dataclass
from pathlib import Path
from deepFakeDetection.constants import *
from deepFakeDetection.utils.common import read_yaml, create_directories
from deepFakeDetection.entity.config_entity import *

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    spectogram_data_dir: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int



2025-01-04 14:28:41.542667: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from deepFakeDetection.constants import *
from deepFakeDetection.utils.common import read_yaml, create_directories, save_json
from deepFakeDetection.entity.config_entity import *

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifact_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            spectogram_data_dir="../artifacts/data_transformation/for-norm/",
            mlflow_uri="https://dagshub.com/wkarlina001/end-to-end-ml.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )

        return eval_config

In [18]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
from tensorflow.keras.utils import to_categorical


In [23]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def load_images_from_path(self, path, label):
        images = []
        labels = []

        for file in os.listdir(path):
            try:
                images.append(img_to_array(load_img(os.path.join(path, file), target_size=(self.config.params_image_size))))
                labels.append((label))
            except:
                continue
        return images, labels

    def create_dataset(self, subdir):
        print(os.path.join(self.config.spectogram_data_dir, subdir, "real"))
        x = []
        y = []
        images, labels = self.load_images_from_path(os.path.join(self.config.spectogram_data_dir, subdir, "real"), 1)
        x += images
        y += labels

        images, labels = self.load_images_from_path(os.path.join(self.config.spectogram_data_dir, subdir, "fake"), 0)
        x += images
        y += labels
        
        return x, y

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        # self.model = self.load_model(self.config.path_of_model)
        self.model = tf.keras.models.load_model("../artifacts/training/model.h5")
        
        x, y = self.create_dataset("testing")
        x1 = np.array(x) / 255
        y1 = to_categorical(y)

        self.score = self.model.evaluate(x1, y1, batch_size=128)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="CustomCNNModel")
            else:
                mlflow.keras.log_model(self.model, "model")

In [24]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-01-04 15:06:48,063: INFO: common: yaml file: /Users/wiwikkarlina/Documents/Escape_Plan/end-to-end-ml/config/config.yaml loaded successfully]
[2025-01-04 15:06:48,070: INFO: common: yaml file: /Users/wiwikkarlina/Documents/Escape_Plan/end-to-end-ml/params.yaml loaded successfully]
[2025-01-04 15:06:48,073: INFO: common: created directory at: artifacts]
../artifacts/data_transformation/for-norm/testing/real


2025-01-04 15:07:31.484386: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


37/37 [==============================] - 5s 82ms/step - loss: 10.1685 - accuracy: 0.4716
[2025-01-04 15:07:35,388: INFO: common: json file saved at: scores.json]
https


2025/01/04 15:07:36 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-01-04 15:07:37,185: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 4 of 4). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: /var/folders/s3/n_t4f85d55b1c46dx20rkhmc0000gn/T/tmp_zb6m16y/model/data/model/assets
[2025-01-04 15:07:38,157: INFO: builder_impl: Assets written to: /var/folders/s3/n_t4f85d55b1c46dx20rkhmc0000gn/T/tmp_zb6m16y/model/data/model/assets]


Registered model 'CustomCNNModel' already exists. Creating a new version of this model...
2025/01/04 15:08:19 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: CustomCNNModel, version 2
Created version '2' of model 'CustomCNNModel'.
